In [1]:
import pandas as pd
import numpy
from timeit import default_timer as timer
from models import Transaction, Item, Transaction_Item

In [2]:
dtype = {
    '交易id': numpy.str,
    '資料日期': numpy.str,
    '資料時間': numpy.str,
    '餐別帶': numpy.str,
    '縣市別': numpy.str,
    '店舖代號': numpy.uint32,
    '主商圈': numpy.str,
    '品號-品名稱': numpy.str,
    '群號-群名稱': numpy.str,
    '單品名稱': numpy.str,
    '銷售數量': numpy.uint16,
    '銷售單價': numpy.float,
    '交易金額': numpy.float
}

In [3]:
USE_COLUMNS = ['交易id', '資料日期', '資料時間', '餐別帶', '縣市別', '店舖代號', '主商圈', '品號-品名稱',
       '群號-群名稱', '單品名稱', '銷售數量', '銷售單價', '交易金額']
PARSE_DATES = {
    '資料日期與時間': [
        '資料日期',
        '資料時間'
    ]
}

In [7]:
chunk_iterator = pd.read_csv('customer_data(utf-8).csv',
                   index_col=1,
                   chunksize=2000,
                   nrows=50000,
                   usecols=USE_COLUMNS,
                   dtype=dtype,
                   parse_dates=PARSE_DATES,
        )

In [8]:
def extract_transaction(index, row):
    return (
        ('id', index),
        ('time', row['資料日期與時間']),
        ('time_phase', row['餐別帶']),
        ('branch_id', row['店舖代號']),
        ('location_type', row['主商圈']),
        ('location', row['縣市別'])
    )

def extract_item(row):
    return (
        ('type', row['品號-品名稱']),
        ('subtype', row['群號-群名稱']),
        ('name', row['單品名稱']),
        ('price', row['銷售單價'])
    )

def extract_transaction_item(index, row):
    return (
        ('transaction_id', index),
        ('item_name', row['單品名稱']),
        ('times', row['銷售數量']),
        ('transaction_amount', row['交易金額'])
    )

In [9]:
datas = pd.read_csv('customer_data(utf-8).csv',
                   index_col=1,
                   nrows=50000,
                   usecols=USE_COLUMNS,
                   dtype=dtype,
                   parse_dates=PARSE_DATES,
        )

In [31]:
datas

,資料日期與時間,餐別帶,縣市別,店舖代號,主商圈,品號-品名稱,群號-群名稱,單品名稱,銷售數量,銷售單價,交易金額
交易id,,,,,,,,,,,
01637920171201184556000112534201,2017-12-01 18:45:56,晚餐時間帶,台中市,11148,住宅型,72-家庭雜貨,722-家雜用品,（新）銷售用購物袋１８號袋,1,1.0,1.0
01564720171201184641000113754802,2017-12-01 18:46:41,晚餐時間帶,台中市,3860,住宅型,00-傳統代收,11-信用卡,花旗信用卡,1,0.0,0.0
01614720171201163042000114405402,2017-12-01 16:30:42,下午茶時間帶,台中市,10697,住宅型,29-冷藏飲料,296-冷藏奶茶,飲冰室茶集綠奶茶,1,25.0,25.0
01614720171201072745000114390002,2017-12-01 07:27:45,早餐時間帶,台中市,10697,住宅型,29-冷藏飲料,296-冷藏奶茶,義美奶茶,1,35.0,35.0
01614720171201065809000118172701,2017-12-01 06:58:09,早餐時間帶,台中市,10697,住宅型,28-其他乳品,286-發酵乳,養樂多,1,8.0,8.0
01614720171201065009000114387302,2017-12-01 06:50:09,早餐時間帶,台中市,10697,住宅型,27-鮮乳,271-大包裝全脂鮮乳,瑞穗全脂鮮乳９３０ｍｌ,1,87.0,87.0
01384020171201130558000119295301,2017-12-01 13:05:58,午餐時間帶,彰化縣,13840,文教型,12-調理麵,122-熱食麵２配,沙茶牛肉炒麵,1,69.0,69.0
01384020171201130034000119295201,2017-12-01 13:00:34,午餐時間帶,彰化縣,13840,文教型,68-保健衛生,682-醫療用品,醫療平面口罩１０入,1,79.0,79.0
01384020171201114808000119291701,2017-12-01 11:48:08,午餐時間帶,彰化縣,13840,文教型,61-啤酒,612-進口啤酒,麒麟霸啤酒３３０ＭＬ,3,32.0,88.0


In [28]:
def get_transaction_dict(df):
    transaction_df = df.drop(['品號-品名稱', '群號-群名稱', '單品名稱', '銷售數量', '銷售單價'], axis=1)
    groupbyObject = transaction_df.groupby(['交易id'])
    transaction_df = groupbyObject.agg({ '資料日期與時間': 'first', '餐別帶': 'first', '縣市別': 'first', '店舖代號': 'first', '主商圈': 'first', '交易金額': 'sum'})
    return transaction_df.to_dict('index')

In [60]:
def get_items_dict(df):
    items = df.set_index('單品名稱')
    items = items.drop(['餐別帶', '資料日期與時間', '縣市別', '店舖代號', '主商圈', '銷售數量', '交易金額'], axis=1)
    return items.groupby(items.index).first().to_dict('index')

In [61]:
items = get_items_dict(datas)

In [62]:
items

{'一口爆漿餡餅（５入）': {'品號-品名稱': '24-冷凍食品', '群號-群名稱': '241-冷凍調理', '銷售單價': 28.0},
 '一度贊爌肉麵（碗）': {'品號-品名稱': '43-泡麵', '群號-群名稱': '433-大碗麵', '銷售單價': 53.0},
 '一度贊爌肉麵（袋）': {'品號-品名稱': '43-泡麵', '群號-群名稱': '435-袋麵', '銷售單價': 47.0},
 '一度贊麻辣牛肉麵（碗）': {'品號-品名稱': '43-泡麵', '群號-群名稱': '433-大碗麵', '銷售單價': 53.0},
 '一日水果１００％蘋果汁': {'品號-品名稱': '29-冷藏飲料', '群號-群名稱': '297-冷藏常溫飲料', '銷售單價': 20.0},
 '一日蔬果１００％紫色蔬果汁': {'品號-品名稱': '29-冷藏飲料', '群號-群名稱': '297-冷藏常溫飲料', '銷售單價': 20.0},
 '一日蔬果１００％蔬果汁': {'品號-品名稱': '29-冷藏飲料', '群號-群名稱': '297-冷藏常溫飲料', '銷售單價': 20.0},
 '一條根精油貼布（薑黃）': {'品號-品名稱': '68-保健衛生', '群號-群名稱': '683-保健用品', '銷售單價': 139.0},
 '一產保險費': {'品號-品名稱': '00-傳統代收', '群號-群名稱': '05-產險', '銷售單價': 0.0},
 '一番榨生啤酒－５００': {'品號-品名稱': '61-啤酒', '群號-群名稱': '612-進口啤酒', '銷售單價': 60.0},
 '一銀一般內': {'品號-品名稱': '00-傳統代收', '群號-群名稱': '14-銀行分期款', '銷售單價': 0.0},
 '一銀信用卡': {'品號-品名稱': '00-傳統代收', '群號-群名稱': '11-信用卡', '銷售單價': 0.0},
 '一顆蛋蛋糕': {'品號-品名稱': '19-吐司蛋糕', '群號-群名稱': '192-蛋糕', '銷售單價': 28.0},
 '七七乳加經濟包': {'品號-品名稱': '36-巧克力', '群號-群名稱': '364-兒童巧克力', '銷售單價': 50.0